<a href="https://colab.research.google.com/github/laguz/DS-Unit-2-Applied-Modeling/blob/master/LS_DS_415_Sprint_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [98]:
import pandas as pd

yelp = pd.read_json('/content/sample_data/review_sample.json', lines=True)

     |████████████████████████████████| 827.9MB 1.1MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp37-none-any.whl size=829180944 sha256=89756062e6065df5031ba41e239eccb18690bdfbc07dd5f474a5dfd549fae2cf
  Stored in directory: /tmp/pip-ephem-wheel-cache-cc9n8ju4/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [3]:
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [94]:
# Import the library
import spacy, re

# Get the text from the DataFrame
review_raw = yelp['text']
review_raw = review_raw[:10]

# Removing the numbers
i = 0
review_raw_num = []
for text in review_raw:
  text = re.sub('[^a-zA-Z .]', '', review_raw[i])
  review_raw_num.append(text)
  i += 1

# Convert to a list and string
review_raw_list = list(review_raw_num)
review_raw_str = str(review_raw_list)


# Load the language library
nlp = spacy.load("en_core_web_sm")
nlp.max_length = 7000000
doc = nlp(review_raw_str)

# Initialize a list to hold the tokens
tokens = []

# Loop over each token in the document (doc)
for token in doc:
    if (token.is_stop == False) & (token.is_punct == False):
            tokens.append((token.lemma_).lower())

# Print the first 50 tokens      
print(tokens[:50])

['beware', 'fake', 'fake', 'fake', 'small', 'business', 'los', 'alamitos', 'receive', 'look', 'like', 'legitimate', 'bill', ' ', 'account', 'number', ' ', 'call', 'phone', 'number', 'list', ' ', ' ', 'wait', 'time', 'hold', 'say', ' ', 'minute', 'leave', 'message', ' ', 'live', 'person', 'phone', 'matter', 'number', 'select', ' ', 'leave', 'firm', 'message', 'contact', 'bbb', 'attorney', 'company', 'try', 'scam', 'business', 'illegal']


## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [147]:
# Create the vectors for each document
from sklearn.feature_extraction.text import TfidfVectorizer

# Instantiate vectorizer object
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)

# Create a vocabulary and get tfidf values per document
dtm = tfidf.fit_transform(review_raw)

# Imports
import pandas as pd

# Get feature names to use as DataFrame column headers
dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())

In [177]:
from sklearn.neighbors import NearestNeighbors
rw = ["Very nice and clean eating area. I was waiting for my car to get fixed and walked over for lunch. I was greeted by a very nice, very sweet lady."]
rw = tfidf.transform(rw)
nn = 10
KNN = NearestNeighbors(nn, p=2)
KNN.fit(dtm)
NNs = KNN.kneighbors(rw, return_distance=True)
NNs

(array([[1.30351012, 1.33371448, 1.34141087, 1.37782833, 1.38877127,
         1.41421356, 1.41421356, 1.41421356, 1.41421356, 1.41421356]]),
 array([[1, 2, 4, 9, 5, 0, 3, 6, 7, 8]]))

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

In [206]:
# Create the DataFrame of sentence and label
df = pd.DataFrame(columns=['sentence', 'label'])
df['sentence'] = yelp['text']
df['label'] = yelp['stars']
df.head()

,sentence,label
0,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",1
1,Came here for lunch Togo. Service was quick. S...,4
2,I've been to Vegas dozens of times and had nev...,3
3,We went here on a night where they closed off ...,1
4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",4


In [213]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression

# Create the features and target
sentences = df['sentence'][:100]
y = df['label'][:100]

# Instantiate the tf-idf vectorizer
vectorizer = TfidfVectorizer(stop_words='english', ngram_range = (2,2))

# Instantiate the classifier (defaults)
classifier = LogisticRegression(solver='lbfgs')

# Instantiate the LSA (SVD) algorithm (defaults)
svd = TruncatedSVD()

In [214]:
# Create the pipelines
from sklearn.pipeline import Pipeline

# LSA part
lsa = Pipeline([('vect', vectorizer), ('svd', svd)])

# Combine into one pipeline
pipe = Pipeline([('lsa', lsa), ('clf', classifier)])

# Define the parameter space for the grid search
parameters = {
    'lsa__svd__n_components': (100,250),
    'lsa__vect__max_df': (0.9, 1.0), # max document frequency
}

# Implement a grid search with cross-validation
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(sentences, y);

# Display the best score from the grid-search
grid_search.best_score_

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    3.5s finished


0.43

## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
    - Keep the `iterations` parameter at or below 5 to reduce run time
    - The `workers` parameter should match the number of physical cores on your machine.
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

In [220]:
from gensim.models import LdaMulticore
from gensim.corpora import Dictionary

In [230]:
# Imports
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

# Add additional stop words
#STOPWORDS = set(STOPWORDS).union(set(['said', 'mr', 'mrs']))

# Function for tokenizing the 
def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

# Split the string on the newline character
text = sentences

# Tokenize each chunk of text
text_tokens = [tokenize(chunk) for chunk in text]

# Look at the first 10 tokens
text_tokens[0][0:10]

['beware',
 'fake',
 'fake',
 'fake',
 'small',
 'business',
 'los',
 'alamitos',
 'ca',
 'received']

In [226]:
# Imports
#from gensim import corpora

# Create the term dictionary of our corpus
# every unique term is assigned an index
dictionary = Dictionary(text_tokens)

# Convert list of documents (corpus) into Document Term Matrix 
# using the dictionary we just created
doc_term_matrix = [dictionary.doc2bow(doc) for doc in text_tokens]

# What does this matrix look like?
print(doc_term_matrix[0][0:25])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 3), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 2)]


In [231]:
# Imports
from gensim.models.ldamulticore import LdaMulticore

# Create the term dictionary of our corpus
# every unique term is assigned an index
dictionary = Dictionary(text_tokens)

# Convert list of documents (corpus) into Document Term Matrix 
# using the dictionary we just created
doc_term_matrix = [dictionary.doc2bow(doc) for doc in text_tokens]

# What does this matrix look like?
print(doc_term_matrix[0][0:25])

# Create the object for LDA model
lda = gensim.models.ldamodel.LdaModel

# Train LDA model on the document term matrix
# topics = 5
ldamodel = lda(doc_term_matrix, num_topics=5, id2word = dictionary, passes=50)

# Print out the topics
print(ldamodel.print_topics(num_topics=3, num_words=5))

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 3), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 2)]
[(2, '0.032*"circus" + 0.027*"room" + 0.016*"fake" + 0.016*"number" + 0.011*"night"'), (1, '0.013*"long" + 0.013*"fabulous" + 0.013*"perfection" + 0.013*"way" + 0.013*"guys"'), (0, '0.028*"nail" + 0.017*"shop" + 0.017*"pedicure" + 0.012*"ve" + 0.012*"minutes"')]


Learn the vocubalary of the yelp data:

In [229]:
#id2word = ...
# Imports
from gensim.models.ldamulticore import LdaMulticore

# Create the object for LDA model
lda = gensim.models.ldamodel.LdaModel

# Train LDA model on the document term matrix
# topics = 5
ldamodel = lda(doc_term_matrix, num_topics=5, id2word = dictionary, passes=50)

# Print out the topics
print(ldamodel.print_topics(num_topics=3, num_words=5))

[(1, '0.003*"street" + 0.003*"people" + 0.003*"guys" + 0.003*"came" + 0.003*"busy"'), (0, '0.025*"nail" + 0.016*"shop" + 0.016*"pedicure" + 0.011*"long" + 0.011*"getting"'), (2, '0.015*"came" + 0.015*"service" + 0.015*"tea" + 0.015*"parking" + 0.015*"togo"')]


Create a bag of words representation of the entire corpus

In [242]:
#corpus = ...
# Repeat the topic model from the previous objective
# Imports
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

# Add additional stop words
#STOPWORDS = set(STOPWORDS).union(set(['said', 'mr', 'mrs']))

# Function for tokenizing the 
def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

# Read in the text (download to run locally)
text_str = sentences

# Split the string on the newline character
#text = text_str.split('\n')

# Tokenize each chunk of text
text_tokens = [tokenize(chunk) for chunk in text]

# Imports
from gensim import corpora

# Create the term dictionary of our corpus
# every unique term is assigned an index
dictionary = corpora.Dictionary(text_tokens)

# Convert list of documents (corpus) into Document Term Matrix 
# using the dictionary we just created
doc_term_matrix = [dictionary.doc2bow(doc) for doc in text_tokens]

# Imports
from gensim.models.ldamulticore import LdaMulticore

# Create the object for LDA model
lda = gensim.models.ldamodel.LdaModel

# Train LDA model on the document term matrix
# topics = 5
ldamodel = lda(doc_term_matrix, num_topics=5, id2word = dictionary, passes=50)

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

Your LDA model should be ready for estimation: 

In [ ]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   iterations=5,
                   workers=-1,
                   num_topics = 10 # You can change this parameter
                  )

Create 1-2 visualizations of the results

## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Create more visualizations of the LDA results and provide written analysis
* Incorporate RandomizedSearchCV into docoument classification pipeline
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)